# Search Engine With Tools and Agents

In [1]:
!pip install wikipedia


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=45f8071243b23d3804a14b94398c73339ee91b20f11010b88193a4ac232d86cb
  Stored in directory: c:\users\kumar\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [2]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=21438c39a4c934272efe10b2c199d8ba19aa2c846ae74059a3b99e778eb38652
  Stored in directory: c:\users\kumar\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [19]:
## Importing tools and API Wrapper 
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import  ArxivAPIWrapper,WikipediaAPIWrapper

In [20]:
# Used the inbuild tools of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
# Making the wikipedia tool with name wiki
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki


WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\kumar\\anaconda3\\envs\\ann_project\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [21]:
# Used the inbuild tools of Arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
# Making the Arxiv tool with name arxiv
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [22]:
# Make the list of tools
tools = [wiki,arxiv]

In [23]:
#Make the custom Tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:
import os
from dotenv import load_dotenv
# Load all the environment variables
load_dotenv()

True

In [25]:
os.environ['HF_TOKEN'] = os.getenv("HUGGING_FACE_TOKEN")

In [29]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

In [31]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.9 MB 5.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/14.9 MB 5.6 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.9 MB 3.7 MB/s eta 0:00:04
   -------- ------------------------------- 3.1/14.9 MB 4.5 MB/s eta 0:00:03
   --------- ------------------------------ 3.4/14.9 MB 3.7 MB/s eta 0:00:04
   ------------- -------------------------- 5.0/14.9 MB 4.2 MB/s eta 0:00:03
   --------------- ------------------------ 5.8/14.9 MB 4.2 MB/s eta 0:00:03
   ---------------- ----------------------- 6.0/14.9 MB 3.8 MB/s eta 0:00:03
   ------------------- -------------------- 7.1/14.9 MB 3.9 MB/s eta 0:00:03
   ------------------- -------------------- 7.3/14.9 MB 3.6 MB/s eta 0:00:03
   --------------------- ------------------ 8.1/14.9 MB 3.7 MB/s eta 0:00:02
   ----------------------- ---------------- 8.7/14.9 MB 3.7 MB/s eta 0:00:02
   ---

In [33]:

#Making the documents 
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
#Store in the vetor Database 
vectordb = FAISS.from_documents(documents,HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2"))
#Making the retriever from the vector database
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020B82237560>, search_kwargs={})

In [34]:
## Convert retriever as a tool

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [35]:
tools = [wiki, arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\kumar\\anaconda3\\envs\\ann_project\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000020BD90D0F40>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000

In [37]:
from langchain_groq import ChatGroq

In [36]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [39]:
llm = ChatGroq(groq_api_key = groq_api_key,model_name = "Llama3-8b-8192")
llm
# Run all this tools with Agents and LLM Models


## Tools, LLM--> AgentsExecutor

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020B86BC1550>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020B86D89BE0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [41]:
## Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [42]:
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [43]:
## Agent Executor 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [47]:
chat = agent_executor.invoke({"input":"Tell me about Langsmith"})
chat



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Log your first trace​Tracing to LangSmith for LangChain usersThere is no need to use the LangSmith SDK directly if your application is built entirely on LangChain (either Python and JS).We've outlined a tracing guide specifically for LangChain users here.We provide multiple ways to log traces to LangSmith. Below, we'll highlight

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Cre

{'input': 'Tell me about Langsmith',
 'output': 'It looks like the tool call "call_hf4z" yielded some documentation about LangSmith. It seems that LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, making it easier to ship quickly and with confidence.\n\nAccording to the documentation, LangSmith can be used without the LangChain library, and it provides multiple ways to log traces to LangSmith. The documentation also provides a quick start guide and tutorials on how to get started with LangSmith.\n\nThe documentation includes examples in both Python and TypeScript, showing how to install LangSmith, create an API key, set up the environment, and log the first trace. It also provides examples of how to use traceable to auto-trace LLM calls in-context.\n\nI\'ll make sure to keep this information in mind as we continue our conversation.'}

In [49]:
chat['output']

'It looks like the tool call "call_hf4z" yielded some documentation about LangSmith. It seems that LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows developers to closely monitor and evaluate their applications, making it easier to ship quickly and with confidence.\n\nAccording to the documentation, LangSmith can be used without the LangChain library, and it provides multiple ways to log traces to LangSmith. The documentation also provides a quick start guide and tutorials on how to get started with LangSmith.\n\nThe documentation includes examples in both Python and TypeScript, showing how to install LangSmith, create an API key, set up the environment, and log the first trace. It also provides examples of how to use traceable to auto-trace LLM calls in-context.\n\nI\'ll make sure to keep this information in mind as we continue our conversation.'

In [50]:
agent_executor.invoke({"input":"What is the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" introduced the Transformer architecture, which has since become widely used in natural language processing tasks.

> Finished chain.


{'input': 'What is the paper 1706.03762 about?',
 'output': 'The paper "Attention Is All You Need" introduced the Transformer architecture, which has since become widely used in natural language processing tasks.'}